In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
with open("key.txt", "r") as file:
    key = file.read()

In [8]:
key = key.strip("\n")

In [9]:
key

'_6GbtoEuFwIC3sfBrrEv4ZJkehWHBSE7jGR6nclrqOxkQUvxsP99XQL-v7l0sxV5J4wT7gEId-VaG9XzMBcyK1mQgvOQoaJnuU61xX2ebQ5bxEPnjruAnr1TgwvJZHYx"'

In [10]:
url = "https://api.yelp.com/v3/businesses/search?location=berlin&term=food&categories=italian%2C%20pizza"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {key}"
}

response = requests.get(url, headers=headers)



In [11]:
resp_dict = response.json()

In [12]:
type(resp_dict)

dict

In [10]:
# empty dict of businesses with key "name", "rating", "zip code"
restaurants = {"name": [], "rating":[], "zip_code": []}


In [11]:
for ent in resp_dict["businesses"]:
    restaurants["name"].append(ent["alias"])
    restaurants["rating"].append(ent["rating"])
    restaurants["zip_code"].append(ent["location"]["zip_code"])

In [8]:
resp_dict.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
type(resp_dict["businesses"])

list

In [12]:
restaurants["name"]

['fontana-di-trevi-berlin',
 'mozzarella-und-pomodoro-berlin-2',
 'tra-di-noi-berlin',
 'sapori-di-casa-berlin',
 'ristorante-a-mano-berlin',
 'buffalo-grill-berlin',
 'café-journale-berlin',
 'pizza-espresso-berlin',
 'dolcini-berlin',
 'pizza-el-fredo-berlin',
 'caramel-berlin-2',
 'osteria-caruso-berlin',
 'simela-berlin-3',
 'ristorante-primavera-berlin',
 'oliosalepepe-berlin',
 'pizza-de-lux-berlin',
 '39-piùtrentanove-berlin-2',
 'sorrentina-berlin',
 'mamma-leone-berlin-2',
 'mani-in-pasta-berlin-2']

In [13]:
df_rest = pd.DataFrame.from_dict(restaurants)

In [14]:
df_rest.head()

,name,rating,zip_code
0,fontana-di-trevi-berlin,4.0,10117
1,mozzarella-und-pomodoro-berlin-2,5.0,10119
2,tra-di-noi-berlin,4.5,10785
3,sapori-di-casa-berlin,5.0,10559
4,ristorante-a-mano-berlin,4.5,10243


In [17]:
url = "https://www.berlinstadtservice.de/xinh/Postleitzahlen_Berlin_Alphabetisch.html"
resp_scrape = requests.get(url)

soup = BeautifulSoup(resp_scrape.content, "html.parser")

In [29]:
table_entries = soup.find_all("td")

In [43]:
zip_codes = []

In [44]:
for ent in table_entries:
    zip_codes.append(ent.text)

In [42]:
zip_codes

['12487 B- Adlershof',
 '12489 B- Adlershof',
 '12524 B- Altglienicke',
 '12435 B- Alt-Treptow',
 '12437 B- Baumschulenweg',
 '12683 B- Biesdorf',
 '13129 B-Blankenburg',
 '13159 B- Blankenfelde',
 '12526 B- Bohnsdorf',
 '13509 B- Borsigwalde',
 '12099 B- Britz',
 '12347 B- Britz',
 '12349 B- Britz',
 '12351 B- Britz',
 '12359 B- Britz',
 '13125 B- Buch',
 '13127 B- Buchholz',
 '12107 B-Buckow',
 '12305 B-Buckow',
 '12349 B-Buckow',
 '12351 B-Buckow',
 '12353 B-Buckow',
 '12357 B-Buckow',
 '12359 B-Buckow',
 '10585 B-Charlottenburg',
 '10587 B-Charlottenburg',
 '10589 B-Charlottenburg',
 '10623 B-Charlottenburg',
 '10625 B-Charlottenburg',
 '10627 B-Charlottenburg',
 '10629 B-Charlottenburg',
 '10707 B-Charlottenburg',
 '10709 B-Charlottenburg',
 '10711 B-Charlottenburg',
 '10719 B-Charlottenburg',
 '10787 B-Charlottenburg',
 '10789 B-Charlottenburg',
 '13353 B-Charlottenburg',
 '13597 B-Charlottenburg',
 '13627 B-Charlottenburg',
 '13629 B-Charlottenburg',
 '14050 B-Charlottenburg ',


In [51]:
clean_zip = []

In [52]:
for ent in zip_codes:
    ent = ent.replace("B-", "")
    clean_zip.append(ent)

In [54]:
plz = []
localities = []
for ent in clean_zip:
    plz.append(ent.split()[0])
    localities.append(ent.split()[1])

In [55]:
plz

['12487',
 '12489',
 '12524',
 '12435',
 '12437',
 '12683',
 '13129',
 '13159',
 '12526',
 '13509',
 '12099',
 '12347',
 '12349',
 '12351',
 '12359',
 '13125',
 '13127',
 '12107',
 '12305',
 '12349',
 '12351',
 '12353',
 '12357',
 '12359',
 '10585',
 '10587',
 '10589',
 '10623',
 '10625',
 '10627',
 '10629',
 '10707',
 '10709',
 '10711',
 '10719',
 '10787',
 '10789',
 '13353',
 '13597',
 '13627',
 '13629',
 '14050',
 '14052',
 '14053',
 '14055',
 '14057',
 '14059',
 '14169',
 '14193',
 '14195',
 '13057',
 '12527',
 '13127',
 '13158',
 '10827',
 '12157',
 '12159',
 '12161',
 '12163',
 '14197',
 '12587',
 '10243',
 '10245',
 '10247',
 '10249',
 '13465',
 '14089',
 '12527',
 '10711',
 '14055',
 '14193',
 '14199',
 '13587',
 '13503',
 '13505',
 '13089',
 '12619',
 '12621',
 '12627',
 '12629',
 '13465',
 '13467',
 '12589',
 '13051',
 '13053',
 '13055',
 '13057',
 '13059',
 '12487',
 '12489',
 '10318',
 '12527',
 '13125',
 '12619',
 '12621',
 '12623',
 '13505',
 '12555',
 '12557',
 '12559',


In [14]:
wiki_resp = requests.get("https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin")
wiki_soup = BeautifulSoup(wiki_resp.content, "html.parser")

In [15]:
all_tables = wiki_soup.find_all("dl")

In [16]:
all_tables

[<dl><dt>(01) <a href="/wiki/Mitte" title="Mitte">Mitte</a></dt></dl>,
 <dl><dt>(02) <a href="/wiki/Friedrichshain-Kreuzberg" title="Friedrichshain-Kreuzberg">Friedrichshain-Kreuzberg</a></dt></dl>,
 <dl><dt>(03) <a href="/wiki/Pankow" title="Pankow">Pankow</a></dt></dl>,
 <dl><dt>(04) <a href="/wiki/Charlottenburg-Wilmersdorf" title="Charlottenburg-Wilmersdorf">Charlottenburg-Wilmersdorf</a></dt></dl>,
 <dl><dt>(05) <a href="/wiki/Spandau" title="Spandau">Spandau</a></dt></dl>,
 <dl><dt>(06) <a href="/wiki/Steglitz-Zehlendorf" title="Steglitz-Zehlendorf">Steglitz-Zehlendorf</a></dt></dl>,
 <dl><dt>(07) <a href="/wiki/Tempelhof-Sch%C3%B6neberg" title="Tempelhof-Schöneberg">Tempelhof-Schöneberg</a></dt></dl>,
 <dl><dt>(08) <a href="/wiki/Neuk%C3%B6lln" title="Neukölln">Neukölln</a></dt></dl>,
 <dl><dt>(09) <a href="/wiki/Treptow-K%C3%B6penick" title="Treptow-Köpenick">Treptow-Köpenick</a></dt></dl>,
 <dl><dt>(10) <a href="/wiki/Marzahn-Hellersdorf" title="Marzahn-Hellersdorf">Marzahn-He